This file explains how my machine learning algoithm that is used in the MLBacktester class works

Getting and preparing the data

In [ ]:
#Import required libiraes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#Reading the dataset
data = pd.read_csv("dataset.csv", parse_dates=['time'], index_col='time')
data

In [ ]:
#Preparing the data
data["returns"] = np.log(data.div(data.shift(1)))
data.dropna(inplace=True)
data["direction"] = np.sign(data.returns)
data

In [ ]:
lags = 5
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace=True)
data

Predicting the market direction

In [ ]:
lm = LogisticRegression(C = 1e6, max_iter = 100000, multi_class = "ovr")
lm.fit(data[cols], data.direction)
data["pred"] = lm.predict(data[cols])
data

Simple backtesting

In [ ]:
data["strategy"] = data.pred * data.returns
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
data[["creturns", "cstrategy"]].plot(figsize = (12 , 8))
plt.show()

The orange line on the graph graph above shows that the stratergy the machine learning algorithm is implementing would produce a 50% return on investement during the trading year 2019-2020. This however does not take into consideration trading costs. The blue line shows a simple buy and hold startergy to compare to the algorithms statergy

In [ ]:
# This code will display the number of trades that the algorithm executed
data["trades"] = data.pred.diff().fillna(0).abs()
data.trades.value_counts()

This algorithm is showing signs of excessive trading. This could mean that this is not the most cost efficent stratergy becuase the more trades the alogirtm takes, the more comission it will have to pay to the executing broker. I am now going to do an out-sample forward test to see how well this current algroithm generalises to unseen data.

Out-sample forward test

In [ ]:
data = pd.read_csv("test_set.csv", parse_dates = ["time"], index_col = "time")
data["returns"] = np.log(data.div(data.shift(1)))
data["direction"] = np.sign(data.returns)

lags = 5
cols = []
for lag in range(1, lags + 1):
    col = "lag{}".format(lag)
    data[col] = data.returns.shift(lag)
    cols.append(col)
data.dropna(inplace = True)

data["pred"] = lm.predict(data[cols])

# This will calulate how oftern the algorithm is correct when predicting the change in price. Note, this is not the only factor that matters when trading
# You also need to look at the overall risk to reward ratio, which I will do later.
hits = np.sign(data.direction * data.pred).value_counts()
hit_ratio = hits[1.0] / sum(hits)
hit_ratio


The hit ratio for unseen data is very similar to that of the hit ratio of the in sample test, which indicates that we do not have over fitting

In [ ]:
data["strategy"] = data.pred * data.returns
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
data[["creturns", "cstrategy"]].plot(figsize = (12 , 8))
plt.show()

Again the orange line shows the algoirithms performance against the blue lines buy and hold performace. The alogirthm out performs it again and produces a 30% return on investment before trading costs.

In [ ]:
data["trades"] = data.pred.diff().fillna(0).abs()
data.trades.value_counts()

I am happy with how the this simple classification model currently performs but I beleive it could be improved upon further by using a more sofisicated classification model and include contraints that will prevent the model of over trading as the commissions on the these unneeded trade will eat away at the algorithms profits.